In [1]:
import pandas as pd
import numpy as np
import functools as ft
import itertools as it
import multiprocessing as mp
from go_learning_helpers import *
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
file_name = 'kinase_go_annotations.csv'
path_to_dir = './'

# get the go annotations
go_annotations = get_go_annotations_series(path_to_dir+file_name)

In [3]:
# binarize
go_annotations = convert_go_annotations_to_one_hot(go_annotations,
                                                   return_binarizer= False)

In [4]:
consensus_file_name = 'consensusclusters_spinglass_greaterthan90percent.txt'
path_to_consensus = '../reproduced/'

consensus_dat = add_cluster_labels(path_to_consensus+consensus_file_name,
                                  go_annotations)

print(consensus_dat.head(5))

       cluster                                          GO Labels
names                                                            
PAK1         1  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
ERBB2        2  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
RON          3  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
YES          3  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
TYRO3        3  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...


In [5]:
from collections import Counter

c = Counter(consensus_dat['cluster'])

In [6]:
c.most_common(n=10)

[(7, 126),
 (3, 61),
 (6, 58),
 (12, 48),
 (18, 22),
 (28, 22),
 (2, 17),
 (8, 15),
 (44, 9),
 (13, 7)]

In [7]:
from sklearn.svm import SVC
from sklearn.svm import NuSVC
from sklearn.naive_bayes import GaussianNB as GNB
from sklearn.naive_bayes import ComplementNB as CNB

svm = SVC(kernel='linear')
nb = GNB()
cnb = CNB()

classifiers = [svm, nb, cnb]

In [8]:
X_train, X_test, y_train, y_test = get_tts(consensus_dat, test_size=0.3)

for clf in classifiers:
    clf.fit(X_train, y_train);

for clf in classifiers:
    print(score_model(y_test, clf.predict(X_test)))

for clf in classifiers:
    print(score_model(y_test, clf.predict(X_test), 'matthews'))

for clf in classifiers:
    print(score_model(y_test, clf.predict(X_test), 'cohen'))

0.5422535211267606
0.528169014084507
0.5915492957746479
0.45503193724238
0.44407086597559214
0.5196759235400495
0.4500387296669248
0.43890068412361405
0.5169784763356988


In [9]:
nb.theta_[6]

array([0., 0., 0., ..., 0., 0., 0.])

In [10]:
nb.sigma_

array([[2.73071622e-10, 2.73071622e-10, 2.73071622e-10, ...,
        2.73071622e-10, 2.73071622e-10, 2.73071622e-10],
       [2.73071622e-10, 2.73071622e-10, 2.73071622e-10, ...,
        2.73071622e-10, 1.95555556e-01, 2.73071622e-10],
       [2.73071622e-10, 2.73071622e-10, 2.73071622e-10, ...,
        2.73071622e-10, 2.22107441e-02, 2.73071622e-10],
       ...,
       [2.73071622e-10, 2.73071622e-10, 2.73071622e-10, ...,
        2.73071622e-10, 2.73071622e-10, 2.73071622e-10],
       [2.73071622e-10, 2.73071622e-10, 2.73071622e-10, ...,
        2.73071622e-10, 2.73071622e-10, 2.73071622e-10],
       [2.73071622e-10, 2.73071622e-10, 2.73071622e-10, ...,
        2.73071622e-10, 2.73071622e-10, 2.73071622e-10]])

In [11]:
fast_greedy_file_name = 'fastgreedy_clusters.txt'
path_to_fast_greedy = '../reproduced/'

fast_greedy_dat = add_cluster_labels(path_to_fast_greedy+fast_greedy_file_name,
                                  go_annotations)

print(fast_greedy_dat.head(5))

       cluster                                          GO Labels
names                                                            
PAK1         3  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
ERBB2        3  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
RON          6  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
YES          6  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
TYRO3        6  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...


In [12]:
print(fast_greedy_dat['cluster'].nunique())

8


In [13]:
X_train, X_test, y_train, y_test = get_tts(fast_greedy_dat, test_size=0.3)

for clf in classifiers:
    clf.fit(X_train, y_train);

for clf in classifiers:
    print(score_model(y_test, clf.predict(X_test)))

for clf in classifiers:
    print(score_model(y_test, clf.predict(X_test), 'matthews'))

for clf in classifiers:
    print(score_model(y_test, clf.predict(X_test), 'cohen'))

0.5563380281690141
0.5211267605633803
0.6056338028169014
0.42022693906759623
0.37313629882230914
0.4965813974774405
0.41429880843263067
0.3688063799189436
0.49504699009398023


In [14]:
eigen_file_name = 'consensusclusters_leadingeigenvector_greaterthan90percent.txt'
path_to_eigen = '../reproduced/'

eigen_dat = add_cluster_labels(path_to_eigen+eigen_file_name,
                                  go_annotations)

print(eigen_dat.head(5))

       cluster                                          GO Labels
names                                                            
PAK1         1  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
ERBB2        2  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
RON          1  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
YES          1  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
TYRO3        1  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...


In [15]:
print(eigen_dat['cluster'].nunique())

11


In [16]:
X_train, X_test, y_train, y_test = get_tts(eigen_dat, test_size=0.3)

for clf in classifiers:
    clf.fit(X_train, y_train);

for clf in classifiers:
    print(score_model(y_test, clf.predict(X_test)))

for clf in classifiers:
    print(score_model(y_test, clf.predict(X_test), 'matthews'))

for clf in classifiers:
    print(score_model(y_test, clf.predict(X_test), 'cohen'))

0.6619718309859155
0.6338028169014085
0.5492957746478874
0.1203946859156245
0.1644538614030727
0.17056091647220434
0.11926605504587151
0.15899772209567198
0.16154626810591388


In [17]:
from sklearn.metrics import adjusted_rand_score

'ARI is symmetric, so labelings that have pure clusters with members coming from the same classes but unnecessary splits are penalized:'

In [18]:
print('Adj Rand sim between consensus and fast_greedy: ', adjusted_rand_score(labels_pred=consensus_dat['cluster'],labels_true=fast_greedy_dat['cluster']))
print('Adj Rand sim between consensus and leading_eigen: ', adjusted_rand_score(labels_pred=consensus_dat['cluster'],labels_true=eigen_dat['cluster']))
print('Adj Rand sim between eigen and fast_greedy: ', adjusted_rand_score(labels_pred=eigen_dat['cluster'],labels_true=fast_greedy_dat['cluster']))


Adj Rand sim between consensus and fast_greedy:  0.5611680241884158
Adj Rand sim between consensus and leading_eigen:  0.06263246814221338
Adj Rand sim between eigen and fast_greedy:  0.08359260686075506


In [19]:
fast_greedy_c = Counter(fast_greedy_dat['cluster'])
consensus_c = Counter(consensus_dat['cluster'])

In [20]:
print(consensus_c.most_common(8))
print(fast_greedy_c.most_common(8))

[(7, 126), (3, 61), (6, 58), (12, 48), (18, 22), (28, 22), (2, 17), (8, 15)]
[(4, 169), (3, 79), (2, 77), (1, 60), (6, 41), (5, 38), (8, 5), (7, 4)]


In [21]:
len(list(filter(lambda x: x[1] > 1, consensus_c.items())))

28

Let's try seeing how the 

In [24]:
validate_learnability(n_run = 100, dat_df=consensus_dat, clf=nb, parallel=3, metrics=['accuracy', 'matthews'], test_size=0.3)

[(0.5563380281690141, 0.4746253373928352),
 (0.528169014084507, 0.4570130483357137),
 (0.5, 0.4260579712866458),
 (0.5070422535211268, 0.4266250980566518),
 (0.4647887323943662, 0.3832147159182527),
 (0.4859154929577465, 0.412992050145003),
 (0.5070422535211268, 0.4219290731138329),
 (0.528169014084507, 0.4529457106615545),
 (0.43661971830985913, 0.36759988478219197),
 (0.4859154929577465, 0.40792591455419913),
 (0.5070422535211268, 0.4207641936923265),
 (0.4788732394366197, 0.40040293855303266),
 (0.44366197183098594, 0.36362926948862934),
 (0.4859154929577465, 0.39882834611134166),
 (0.5633802816901409, 0.4838824580578401),
 (0.4859154929577465, 0.4129222759321008),
 (0.5492957746478874, 0.4858963175876966),
 (0.5633802816901409, 0.47992095210450053),
 (0.5422535211267606, 0.45816504723044915),
 (0.5211267605633803, 0.443457545077491),
 (0.528169014084507, 0.44489833820992125),
 (0.4859154929577465, 0.40916658183910126),
 (0.49295774647887325, 0.4141005977934184),
 (0.507042253521126